<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing, Evaluation, and Analysis of an LLM Application using Phoenix</h1>

In this tutorial we will learn how to build, observe, evaluate, and analyze a LLM powered application. This is a LLM-powered Chat on Docs application that will answer questions about <a href="https://docs.arize.com/arize/">Arize</a> from their product documentation.

Key Concepts:
1. LLM Traces are  a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context (such as retrieval from vector stores, usage of external tools, etc).

2. Traces are made up of a sequence of `spans`. A span represents a unit of work or operation (think a span of time).

3. LLM Evaluations help get visbility into the performance of the application

Run the next two code blocks to launch Phoenix.

## Launch Phoenix to visualize the app

In [ ]:
%pip install -qq arize-phoenix openai gcsfs nest_asyncio llama-index-llms-openai 'httpx<0.28'

In [ ]:
import os
from getpass import getpass

# Uses your OpenAI API Key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from gcsfs import GCSFileSystem
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from openinference.instrumentation import suppress_tracing
from tqdm import tqdm

## Learn how to Build the application and add Tracing

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

tracer_provider = register(project_name="llm-rag-app")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

In [ ]:
# Pulls down the Arize Product Documentation Knowledge Base
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
index = load_index_from_storage(
    storage_context,
)
query_engine = index.as_query_engine()

# Asking the Application questions about the Arize product
queries = [
    "How can I query for a monitor's status using GraphQL?",
    "How do I delete a model?",
    "How much does an enterprise license of Arize cost?",
    "How do I log a prediction using the python SDK?",
]

for query in tqdm(queries):
    response = query_engine.query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")

## Learn how to Evaluate the application using Phoenix LLM Evals

We now have visibility into the inner workings of our application. Next, let's take a look at how to use LLM evals to evaluate our application.

We will be going through a few common evaluation metrics

1. Hallucination Eval: Checks if application response was an hallucination

2. Q&A Eval: Whether or not the application answers the question correctly

3. Document Relevance Eval: Grades whether or not the documents/chunks retrieved were actually relevant to answering the query

In [ ]:
# Convert traces into workable datasets
from phoenix.client import AsyncClient

px_client = AsyncClient()

spans_df = px_client.get_spans_dataframe(project_name="llm-rag-app")
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px_client, project_name="llm-rag-app")
queries_df = get_qa_with_reference(px_client, project_name="llm-rag-app")

In [ ]:
hallucination_prompt = """
In this task, you will be presented with a query, a reference text and an answer. The answer is
generated to the question based on the reference text. The answer may contain false information. You
must use the reference text to determine if the answer to the question contains false information,
if the answer is a hallucination of facts. Your objective is to determine whether the answer text
contains factual information and is not a hallucination. A 'hallucination' refers to
an answer that is not based on the reference text or assumes information that is not available in
the reference text. Your response should be a single word: either "factual" or "hallucinated", and
it should not include any other text or characters. "hallucinated" indicates that the answer
provides factually inaccurate information to the query based on the reference text. "factual"
indicates that the answer to the question is correct relative to the reference text, and does not
contain made up information. Please read the query and reference text carefully before determining
your response.

    [BEGIN DATA]
    ************
    [Query]: {input}
    ************
    [Reference text]: {reference}
    ************
    [Answer]: {output}
    ************
    [END DATA]

    Is the answer above factual or hallucinated based on the query and reference text?
"""

qa_correctness_prompt = """
You are given a question, an answer and reference text. You must determine whether the
given answer correctly answers the question based on the reference text. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Reference]: {reference}
    ************
    [Answer]: {output}
    [END DATA]
Please read the query, reference text and answer carefully, then write out in a step by step manner
an EXPLANATION to show how to determine if the answer is "correct" or "incorrect". Avoid simply
stating the correct answer at the outset. Your response LABEL must be a single word, either
"correct" or "incorrect", and should not contain any text or characters aside from that word.
"correct" means that the question is correctly and fully answered by the answer.
"incorrect" means that the question is not correctly or only partially answered by the
answer.

Example response:
************
EXPLANATION: An explanation of your reasoning for why the label is "correct" or "incorrect"
LABEL: "correct" or "incorrect"
************

EXPLANATION:"""

In [ ]:
# Generating the Hallucination & Q&A Eval
from phoenix.evals import (
    async_evaluate_dataframe,
    create_classifier,
)
from phoenix.evals.llm import LLM

llm = LLM(provider="openai", model="gpt-4")

hallucination_eval = create_classifier(
    name="hallucination",
    prompt_template=hallucination_prompt,
    llm=llm,
    choices={"factual": 1.0, "hallucinated": 0.0},
)
qa_correctness_eval = create_classifier(
    name="qa_correctness",
    prompt_template=qa_correctness_prompt,
    llm=llm,
    choices={"correct": 1.0, "incorrect": 0.0},
)
with suppress_tracing():
    results_df = await async_evaluate_dataframe(
        dataframe=queries_df,
        evaluators=[hallucination_eval, qa_correctness_eval],
    )
results_df.head()

In [ ]:
from phoenix.evals.utils import to_annotation_dataframe

relevancy_eval_df = to_annotation_dataframe(dataframe=results_df)

await px_client.annotations.log_span_annotations_dataframe(
    dataframe=relevancy_eval_df,
    annotator_kind="LLM",
)

As you can see from the results, one of the queries was flagged as a hallucination.

We can use Retrieval Relevance Evals to identify if these issues are caused by the retrieval process for RAG. We are going to use an LLM to grade whether or not the chunks retrieved are relevant to the query.

In [ ]:
rag_relevancy_prompt = """
You are comparing a reference text to a question and trying to determine if the reference text
contains information relevant to answering the question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Reference text]: {reference}
    ************
    [END DATA]
Compare the Question above to the Reference text. You must determine whether the Reference text
contains information that can answer the Question. Please focus on whether the very specific
question can be answered by the information in the Reference text.
Your response must be single word, either "relevant" or "unrelated",
and should not contain any text or characters aside from that word.
"unrelated" means that the reference text does not contain an answer to the Question.
"relevant" means the reference text contains an answer to the Question."""

In [ ]:
retrieved_documents_eval = create_classifier(
    name="rag_relevancy",
    prompt_template=rag_relevancy_prompt,
    llm=llm,
    choices={"relevant": 1.0, "unrelated": 0.0},
)
with suppress_tracing():
    results_df = await async_evaluate_dataframe(
        dataframe=retrieved_documents_df, evaluators=[retrieved_documents_eval]
    )
results_df.head()

In [ ]:
if results_df.index.name == "context.span_id" and "context.span_id" not in results_df.columns:
    results_df = results_df.reset_index()

document_eval_df = to_annotation_dataframe(dataframe=results_df)

await px_client.annotations.log_span_annotations_dataframe(
    dataframe=document_eval_df,
    annotator_kind="LLM",
)

Looks like we are getting a lot of irrelevant chunks of text that might be polluting the prompt sent to the LLM.

If we once again visit the UI, we will now see that Phoenix has aggregated up retrieval metrics (`precision`, `ndcg`, and `hit`). We see that our hallucinations and incorrect answers directly correlate to bad retrieval!